In [ ]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('data/train.csv').drop(columns=['id', 'lang_abv','language'])
test_df = pd.read_csv('data/test.csv').drop(columns=['id', 'lang_abv', 'language'])                 


In [ ]:
from datasets import Dataset

#Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(train_df, split="train")
test_dataset = Dataset.from_pandas(test_df, split="test")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
# Tokenize both 'premise' and 'hypothesis' columns
def tokenize_function(examples):
    return tokenizer(
        text=[pre + tokenizer.sep_token + hyp for pre, hyp in zip(examples["premise"], examples["hypothesis"])],
        padding="max_length",
        truncation=True
    )

In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()